In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
os.chdir('C:\\Users\\pares\\Downloads\\ds_questions\\ds_questions\\massandopolis')
os.getcwd()

'C:\\Users\\pares\\Downloads\\ds_questions\\ds_questions\\massandopolis'

In [3]:
#what's my data?
all_files = glob.glob('*.csv')
all_files

['allergies.csv',
 'careplans.csv',
 'concepts.csv',
 'conditions.csv',
 'encounters.csv',
 'imaging_studies.csv',
 'immunizations.csv',
 'medications.csv',
 'observations.csv',
 'organizations.csv',
 'patients.csv',
 'patients_updated.csv',
 'procedures.csv',
 'providers.csv']

In [4]:
#loading patients updated file as patients
patients = pd.read_csv('patients_updated.csv')
patients.birthdate = patients.birthdate.apply(pd.to_datetime)
patients.deathdate = patients.deathdate.apply(pd.to_datetime)
patients.head(2)

,id,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,...,address,city,state,zip,eventtime,status,trt,birth_year,death_year,age
0,c0d528b6-cdae-4d98-8936-a270f8827eb7,2013-02-10,NaT,999-83-3170,NaN,NaN,NaN,Claudette391,Hahn503,NaN,...,239 Schaden Glen,Randolph,Massandopolis,2368.0,2.770027,1,0,2013,2050.0,7.0
1,d709f84f-7d2c-440b-a979-2bc27d41f63a,1994-01-10,2019-01-07,999-31-7937,S99989196,X43239863X,Ms.,Christina143,Langworth352,NaN,...,535 Romaguera Annex Unit 82,Boston,Massandopolis,2108.0,2.973888,1,0,1994,2019.0,25.0
